In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/GAN_for_Neural_Graph"

os.chdir(path)
os.listdir(path)

#Import Libaries

In [ ]:
import pandas as pd
import os
import numpy as np
from numpy import expand_dims
from keras import backend as K
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import optimizers
from keras import callbacks
from keras.engine import Layer
from keras.engine import InputSpec
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Add
from keras.utils import to_categorical
from keras.utils import conv_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.initializers import RandomNormal
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

#Load data

In [ ]:
table = pd.read_csv("./Dataset/left_table.csv")
table1 = table.loc[:,['Id','label_id']]
print(table1.head())

In [ ]:
table_dict = dict(zip(table1.Id, table1.label_id))
# print(table_dict)

In [ ]:
csv_files = os.listdir("./Dataset/FC_norm")
csv_files = [file for file in csv_files if file[-1]=='v']
print(len(csv_files))

X = []
y = []

In [ ]:
for file in csv_files:
    df = pd.read_csv(os.path.join("./Dataset/FC_norm",file),header = None)
    X.append(df.to_numpy())
    key = int(file.strip('.csv'))
#     print(key)
    y.append(int(table_dict[key]))

In [ ]:
X = np.array(X)
print(X.shape)
y = np.array(y)
print(y.shape)

In [ ]:
# expand to 3d, e.g. add channels
# convert from ints to floats
X = expand_dims(X, axis=-1).astype('float32')

X, y = shuffle(X, y, random_state=1)
X_train, X_remain, y_train, y_remain = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_remain, y_remain, test_size=0.5, random_state=42)

print(X_train.shape, y_train.shape)  
print(X_val.shape, y_val.shape) 
print(X_test.shape, y_test.shape)

#Hyper parameters

In [ ]:
DROPOUT = 0.5
MOMENTUM = 0.9
LR = 0.001  
DECAY = 0.0005
ALPHA = 0.33
FE_CHANNEL = 64
MERGE_CHANNEL = 32
BATCH_SIZE = 32

'''
The number of E2E layers
'''

#Build Model

In [ ]:
class E2E_conv(Layer):
  def __init__(self, rank,
         filters,
         kernel_size,
         strides=1,
         padding='valid',
         activation=None,
         kernel_initializer='glorot_uniform',
         kernel_regularizer=None,
         kernel_constraint=None,
         **kwargs):
    super(E2E_conv, self).__init__(**kwargs)
    self.rank = rank
    self.filters = filters
    self.kernel_size = conv_utils.normalize_tuple(kernel_size, rank, 'kernel_size')
    self.strides = conv_utils.normalize_tuple(strides, rank, 'strides')
    self.padding = conv_utils.normalize_padding(padding)
    self.activation = activations.get(activation)
    self.kernel_initializer = initializers.get(kernel_initializer)
    self.kernel_regularizer = regularizers.get(kernel_regularizer)
    self.kernel_constraint = constraints.get(kernel_constraint)
    self.input_spec = InputSpec(ndim=self.rank + 2)

  def build(self, input_shape):
    channel_axis = -1
    if input_shape[channel_axis] is None:
      raise ValueError('The channel dimension of the inputs'
               'should be defined. Found `None`.')
    input_dim = input_shape[channel_axis]
    kernel_shape = self.kernel_size + (input_dim, self.filters)

    self.kernel = self.add_weight(shape=kernel_shape,
                    initializer=self.kernel_initializer,
                    name='kernel',
                    regularizer=self.kernel_regularizer,
                    constraint=self.kernel_constraint)
    
    # Set input spec.
    self.input_spec = InputSpec(ndim=self.rank + 2,
                   axes={channel_axis:input_dim})
    self.built = True

  def call(self, inputs):
    kernel_shape = K.get_value(self.kernel).shape
    d = kernel_shape[1]
    kernellxd = K.reshape(self.kernel[0,:], (1, kernel_shape[1], kernel_shape[2], kernel_shape[3]))  # row vector
    kerneldxl = K.reshape(self.kernel[1,:], (kernel_shape[1], 1, kernel_shape[2], kernel_shape[3]))  # column vector
    convlxd = K.conv2d(
        inputs,
        kernellxd,
        strides=self.strides,
        padding=self.padding)
    convdxl = K.conv2d(
        inputs,
        kerneldxl,
        strides=self.strides,
        padding=self.padding)
    concat1 = K.concatenate([convdxl]*d, axis=1)
    concat2 = K.concatenate([convlxd]*d, axis=2)
    return concat1 + concat2

  def compute_output_shape(self, input_shape):
    return (input_shape[0], input_shape[1], input_shape[2], self.filters)

  def get_config(self):
    config = {
        'rank': self.rank,
        'filters': self.filters,
        'kernel_size': self.kernel_size,
        'strides': self.strides,
        'padding': self.padding,
        'activation': activations.serialize(self.activation),
        'kernel_initializer': initializers.serialize(self.kernel_initializer),
        'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
        'kernel_constraint': constraints.serialize(self.kernel_constraint)
    }
    base_config = super(E2E_conv, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

In [ ]:
def define_E2E(in_shape=(200,200,1), n_classes=3):
  # Setting l2_norm regularizer
  reg = regularizers.l2(DECAY)
  # kernel initialization
  kernel_init = initializers.he_uniform()
  in_image = Input(shape=in_shape)

  # E2E layer
  fe = E2E_conv(2, 16, (2, 200), kernel_regularizer=reg)(in_image) 
  fe = LeakyReLU(alpha=ALPHA)(fe)

  fe = E2E_conv(2, 32, (2, 200), kernel_regularizer=reg)(fe) 
  fe = LeakyReLU(alpha=ALPHA)(fe)

  # E2N layer
  temp1 = Conv2D(64, (1, 200), kernel_regularizer=reg, name='row')(fe)   
  temp2 = Conv2D(64, (200, 1), kernel_regularizer=reg, name='column')(fe) 
  temp2 = Reshape((200, 1, 64))(temp2)
  fe = Add()([temp1, temp2])                    
  fe = LeakyReLU(alpha=ALPHA)(fe)

  # N2G layer
  fe = Conv2D(128, (200, 1), kernel_regularizer=reg)(fe)
  fe = LeakyReLU(alpha=ALPHA)(fe)

  # flatten feature maps
  fe = Flatten()(fe)

  fe = Dense(FE_CHANNEL, kernel_regularizer=reg, kernel_initializer=kernel_init)(fe)
  fe = LeakyReLU(alpha=ALPHA)(fe)
  fe = Dropout(DROPOUT)(fe)
  
  merge = Dense(MERGE_CHANNEL, kernel_regularizer=reg, kernel_initializer=kernel_init)(fe)
  merge = LeakyReLU(alpha=ALPHA)(merge)
  merge = Dropout(DROPOUT)(merge)

  out = Dense(n_classes, activation='softmax', name='out')(merge)
    
  # define model
  model = Model(in_image, out)
  return model

In [ ]:
BrainNetCNN = define_E2E()
BrainNetCNN.summary()

# Training

In [ ]:
path = "./BrainNetCNN/mode3/epoch_30.h5"
batch_size = BATCH_SIZE

opt = optimizers.SGD(momentum=MOMENTUM,nesterov=True,lr=LR)
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=0, save_best_only=True)
earlystopping = EarlyStopping(patience=10)
callbacks_list = [checkpoint,earlystopping]

# define model
model_1 = define_E2E()
model_1.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['acc'])

# train
history = model_1.fit(X_train, y_train,
              epochs=30,
              batch_size=batch_size,
              validation_data=(X_val, y_val),
              callbacks=callbacks_list,
              shuffle=True)

In [ ]:
test_data = [X_test, y_test]
test(test_data)

In [ ]:
import matplotlib.pyplot as plt

def plot_learning_curves(history,epoch,min_val,max_val):
        pd.DataFrame(history.history).plot(figsize=(8,5))
        plt.grid(True)
        plt.axis([0, epoch, min_val, max_val])
        plt.show()
plot_learning_curves(history,5,0,2)

# Test

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics

def calculate_score(test, pred):
  new_pred = np.zeros((pred.shape[0], 2))
  new_pred[:, 0] = pred[:, 0]
  new_pred[:, 1] = pred[:, 1] + pred[: ,2]
  score = new_pred[:, 1]
  return score


def test(test_dataset):
  # load model
  path = "./BrainNetCNN/mode3/epoch_30.h5"
  model_1 = define_E2E()
  opt = optimizers.SGD(momentum=MOMENTUM,nesterov=True,lr=LR)
  model_1.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['acc'])
  model_1.load_weights(path)

  X_test, labels_test = test_dataset
  num_test = X_test.shape[0]

  print(f"\nValidation Metrics of Discriminator:")
  test_metrics = model_1.evaluate(X_test, labels_test, verbose=1)
  three_c_acc = 100 * test_metrics[1]

  # two class accuracy
  temp_pred = model_1.predict(X_test)
  labels_pred = np.argmax(temp_pred, axis=1)

  correct = np.sum(labels_pred==labels_test)
  acc = correct / num_test * 100
  print('test: %.3f' % acc)

  labels_2_test, labels_2_pred = labels_test.copy(), labels_pred.copy()
  # classify 2 as 1
  labels_2_test[labels_2_test==2] = 1
  labels_2_pred[labels_2_pred==2] = 1
  # calculate the accuracy
  correct = np.sum(labels_2_test==labels_2_pred)
  two_c_acc = correct / num_test * 100
  print('three class acc: %.3f, two class acc: %.3f' % (three_c_acc, two_c_acc))
  print("="*100)

  # three class confusion metrics
  target_names = ['class 0', 'class 1', 'class 2']
  print('three class:')

  # calculate precision, recall, f1 score
  print(classification_report(labels_test, labels_pred, target_names=target_names, digits=4))

  # calculate AUC
  onehot = to_categorical(labels_test, num_classes=3)
  AUC = metrics.roc_auc_score(onehot, temp_pred, multi_class='ovr')
  print('AUC: ', AUC)
  print("="*100)

  # two class confusion metrics
  target_names = ['class 0', 'class 1']
  print('two class:')
  # calculate precision, recall, f1 score
  print(classification_report(labels_2_test, labels_2_pred, target_names=target_names, digits=4))

  # calcuate specificity
  tn, fp, fn, tp = confusion_matrix(labels_2_test, labels_2_pred).ravel()
  specificity = tn / (tn+fp)
  print('specificity:', specificity)

  # calculate AUC
  score = calculate_score(labels_2_test, temp_pred)
  AUC = metrics.roc_auc_score(labels_2_test, score)
  print('AUC: ', AUC)
  print("="*100)

In [ ]:
test_data = [X_test, y_test]
test(test_data)